In [1]:
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import ConfusionMatrixDisplay, r2_score
from sklearn.utils import shuffle
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib
import torch
import torch.nn as nn

In [2]:
!pip install torch-geometric
!pip install ase
!pip install pymatgen
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 24.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=0964936b3b530c79bccf62fe5cbc2cfdd60fdf9428f3b6867e3b44b4ad4af3a4
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [3]:
import os
import sys
import time
import csv
import json
import warnings
import numpy as np
import ase
import pymatgen as pmg
import glob
from ase import io
from scipy.stats import rankdata
from scipy import interpolate

##torch imports
import torch
import torch.nn.functional as F
from torch_geometric.data import DataLoader, Dataset, Data, InMemoryDataset
from torch_geometric.utils import dense_to_sparse, degree, add_self_loops
import torch_geometric.transforms as T
from torch_geometric.utils import degree

import numpy as np
import json

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/ML4ME/project/3.C51 Project/MatDeepLearn-main')
from matdeeplearn.models.megnet import MEGNet
from matdeeplearn.training.training import trainer, evaluate

In [8]:
# Check if your GPU is requested successfully or not 
assert torch.cuda.device_count() != 0

In [9]:
class StructureDataset(InMemoryDataset):
    def __init__(
        self, data_path, processed_path="processed", transform=None, pre_transform=None
    ):
        self.data_path = data_path
        self.processed_path = processed_path
        super(StructureDataset, self).__init__(data_path, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_dir(self):
        return os.path.join(self.data_path, self.processed_path)

    @property
    def processed_file_names(self):
        file_names = ["data.pt"]
        return file_names


In [10]:
class GetY(object):
    def __init__(self, index=0):
        self.index = index

    def __call__(self, data):
        # Specify target.
        if self.index != -1:
            data.y = data.y[0][self.index]
        return data

In [11]:
data_path = '/content/drive/MyDrive/Colab Notebooks/ML4ME/project/3.C51 Project/MatDeepLearn-main/data/BGML_data/BGML_train_noe/'
processed_path = 'processed'
target_index = 0
transforms = GetY(index=target_index)
if os.path.exists(os.path.join(data_path, processed_path, "data.pt")) == True:
    dataset = StructureDataset(
        data_path,
        processed_path,
        transforms,
    )

In [12]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML4ME/project/3.C51 Project/MatDeepLearn-main/data/BGML_data/MDL_data_AL_test.csv',  header=None)
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML4ME/project/3.C51 Project/MatDeepLearn-main/data/BGML_data/MDL_data_AL_train.csv',  header=None)
data_structure_ids = [x.structure_id[0][0] for x in dataset]
df_data = pd.DataFrame(data_structure_ids)

In [13]:
df_test = df_data[df_data[0].isin(test_df[0])]
test_indices = list(df_test.index)

df_train = df_data[df_data[0].isin(train_df[0])]
train_indices = list(df_train.index)

df_remain = df_data[~df_data[0].isin(df_train[0])]
df_remain = df_remain[~df_remain[0].isin(df_test[0])]
remain_indices = list(df_remain.index)
print(len(test_indices), len(train_indices), len(remain_indices))

2576 4284 11689


In [14]:
df_train_RS = df_train
df_remain_RS = df_remain

In [15]:
train_dataset = torch.utils.data.dataset.Subset(dataset, train_indices)
test_dataset = torch.utils.data.dataset.Subset(dataset, test_indices)
remain_dataset = torch.utils.data.dataset.Subset(dataset, remain_indices)

In [16]:
# test MAE based on initial training set
seed = 0
batch_size = 100
model = MEGNet(dataset).to('cuda')
optimizer = getattr(torch.optim, "AdamW")(
    model.parameters(),
    lr=0.0005
)
scheduler = getattr(torch.optim.lr_scheduler, "ReduceLROnPlateau")(
    optimizer,**{"mode":"min", "factor":0.8, "patience":10, "min_lr":0.00001, "threshold":0.0002}
)
train_ratio = 0.9
val_ratio = 0.1
dataset_size = len(train_dataset)
train_length = int(dataset_size * train_ratio)
val_length = int(dataset_size * val_ratio)
unused_length = dataset_size - train_length - val_length
(
    train_dataset_for_testing,
    val_dataset,
    unused_dataset,
) = torch.utils.data.random_split(
    train_dataset,
    [train_length, val_length, unused_length],
    generator=torch.Generator().manual_seed(seed),
)
train_loader = DataLoader(
        train_dataset_for_testing,
        batch_size=batch_size)
val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size)
test_loader = DataLoader(
    test_dataset,
    batch_size = batch_size)
model = trainer('cuda',1,model,optimizer,scheduler,'l1_loss', train_loader, val_loader, None, 250, 5,"my_model_temp.pth")
test_error, test_out = evaluate(
                test_loader, model, 'l1_loss', 'cuda', out=True
            )

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.09919, Val Error: 1.19825, Time per epoch (s): 2.08034
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.82121, Val Error: 1.00526, Time per epoch (s): 1.80729
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.68730, Val Error: 1.06048, Time per epoch (s): 1.82933
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.58314, Val Error: 0.88077, Time per epoch (s): 1.98056
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.53317, Val Error: 0.92939, Time per epoch (s): 1.89458
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.49055, Val Error: 0.86763, Time per epoch (s): 1.96193
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.42102, Val Error: 0.86044, Time per epoch (s): 1.96459
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.40329, Val Error: 0.82022, Time per epoch (s): 1.88197
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.37697, Val Error: 0.76315, Time per epoch (s): 1.83398
E

In [17]:
test_error_0 = test_error.cpu().numpy()
print('round 0 test error: ', test_error_0)

round 0 test error:  0.94365054


In [18]:
num_folds = 5
dataset_size = len(train_dataset)
fold_length = int(dataset_size / num_folds)
unused_length = dataset_size - fold_length * num_folds
folds = [fold_length for i in range(num_folds)]
folds.append(unused_length)
cv_dataset = torch.utils.data.random_split(
    train_dataset, folds, generator=torch.Generator().manual_seed(seed)
)
cv_dataset = cv_dataset[0:num_folds]

In [19]:
remain_loader = DataLoader(remain_dataset, batch_size=batch_size)

In [20]:
remain_preds_df_all = None

for index in range(0,(len(cv_dataset))):
    train_dataset = [x for i, x in enumerate(cv_dataset) if i != index]
    train_dataset = torch.utils.data.ConcatDataset(train_dataset)
    val_dataset = cv_dataset[index]
    print(len(train_dataset), len(test_dataset))
    train_loader = DataLoader( train_dataset, batch_size=batch_size)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    model = MEGNet(dataset).to('cuda')
    optimizer = getattr(torch.optim, "AdamW")(
        model.parameters(),
        lr=0.0005
    )
    scheduler = getattr(torch.optim.lr_scheduler, "ReduceLROnPlateau")(
        optimizer,**{"mode":"min", "factor":0.8, "patience":10, "min_lr":0.00001, "threshold":0.0002}
    )
    model = trainer('cuda',1,model,optimizer,scheduler,'l1_loss', train_loader, val_loader, None, 50, 5,"my_model_temp.pth")
    remain_error, remain_out = evaluate(
                remain_loader, model, 'l1_loss', 'cuda', out=True
            )
    remain_preds = [[x[0],float(x[2])] for x in remain_out]
    remain_preds_df = pd.DataFrame(remain_preds,columns=('filename','pred'+str(index)))
    if remain_preds_df_all is None:
      remain_preds_df_all = remain_preds_df
    else:
      remain_preds_df_all = pd.merge(remain_preds_df_all, remain_preds_df, on='filename', how='inner')

remain_preds_df_all['unc'] = remain_preds_df_all.std(axis=1,numeric_only=True)
remain_preds_df_all.sort_values('unc',ascending=False, inplace=True)


3424 2576
Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.22172, Val Error: 1.20419, Time per epoch (s): 2.08560
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.87452, Val Error: 1.10723, Time per epoch (s): 1.82200
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.73867, Val Error: 0.94704, Time per epoch (s): 1.75824
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.61802, Val Error: 1.17773, Time per epoch (s): 2.06101
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.58140, Val Error: 0.88665, Time per epoch (s): 1.73633
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.49709, Val Error: 1.43343, Time per epoch (s): 1.73622
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.46404, Val Error: 0.88097, Time per epoch (s): 2.03605
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.44490, Val Error: 0.90605, Time per epoch (s): 1.80468
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.38338, Val Error: 0.83741, Time per epoch (s):

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.20921, Val Error: 1.36362, Time per epoch (s): 2.06588
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.95658, Val Error: 1.16113, Time per epoch (s): 1.90393
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.79954, Val Error: 0.96461, Time per epoch (s): 1.73626
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.67375, Val Error: 0.92259, Time per epoch (s): 1.91165
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.58596, Val Error: 0.97863, Time per epoch (s): 1.77384
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.53434, Val Error: 0.88057, Time per epoch (s): 1.88383
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.47818, Val Error: 0.89563, Time per epoch (s): 1.92610
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.43249, Val Error: 1.00751, Time per epoch (s): 1.88530
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.42993, Val Error: 0.86384, Time per epoch (s): 1.73212
E

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.23503, Val Error: 1.28165, Time per epoch (s): 1.93922
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.93565, Val Error: 1.15320, Time per epoch (s): 1.85641
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.74203, Val Error: 1.15962, Time per epoch (s): 1.77278
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.64885, Val Error: 0.95920, Time per epoch (s): 1.94396
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.56698, Val Error: 0.98573, Time per epoch (s): 1.91477
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.53328, Val Error: 1.00053, Time per epoch (s): 1.79404
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.48068, Val Error: 0.87388, Time per epoch (s): 2.16991
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.47680, Val Error: 0.85840, Time per epoch (s): 1.87458
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.42376, Val Error: 0.84617, Time per epoch (s): 1.74481
E

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.20709, Val Error: 1.42695, Time per epoch (s): 1.95497
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.90079, Val Error: 1.16137, Time per epoch (s): 1.75964
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.72904, Val Error: 1.05108, Time per epoch (s): 1.74969
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.63704, Val Error: 1.01615, Time per epoch (s): 1.98892
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.54231, Val Error: 0.87321, Time per epoch (s): 1.77778
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.52065, Val Error: 0.86719, Time per epoch (s): 1.80169
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.45234, Val Error: 0.89409, Time per epoch (s): 1.95644
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.43884, Val Error: 0.83201, Time per epoch (s): 1.81070
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.48625, Val Error: 1.00436, Time per epoch (s): 1.78160
E

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.20324, Val Error: 1.50683, Time per epoch (s): 1.95481
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.88556, Val Error: 1.28207, Time per epoch (s): 1.88269
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.74085, Val Error: 1.00112, Time per epoch (s): 1.77141
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.60531, Val Error: 0.97221, Time per epoch (s): 1.84758
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.56032, Val Error: 0.85232, Time per epoch (s): 1.75857
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.48817, Val Error: 0.84918, Time per epoch (s): 1.83890
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.44681, Val Error: 0.86424, Time per epoch (s): 1.82681
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.41529, Val Error: 0.89961, Time per epoch (s): 1.76724
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.40238, Val Error: 0.85472, Time per epoch (s): 1.92330
E

In [21]:
add_AL_filenames = pd.DataFrame(remain_preds_df_all['filename'][:500])
add_RS_filenames = pd.DataFrame(df_remain[0].sample(500))

In [22]:
all_AL_test_errors = [float(test_error_0)]
all_RS_test_errors = [float(test_error_0)]

In [ ]:
AL_rounds = 1
while AL_rounds < 20:
  # get new training and remaining data for AL and RS, and create the datasets
  df_train_new = df_data[df_data[0].isin(add_AL_filenames['filename'])]
  df_train = pd.concat((df_train, df_train_new)).sort_index()
  train_indices = list(df_train.index)

  df_train_new_RS = df_data[df_data[0].isin(add_RS_filenames[0])]
  df_train_RS = pd.concat((df_train_RS, df_train_new_RS)).sort_index()
  train_indices_RS = list(df_train_RS.index)

  df_remain = df_data[~df_data[0].isin(df_train[0])]
  df_remain = df_remain[~df_remain[0].isin(df_test[0])]
  df_remain_RS = df_data[~df_data[0].isin(df_train_RS[0])]
  df_remain_RS = df_remain_RS[~df_remain_RS[0].isin(df_test[0])]
  remain_indices = list(df_remain.index)
  remain_indices_RS = list(df_remain_RS.index)

  print(len(test_indices), len(train_indices), len(remain_indices))
  print(len(test_indices), len(train_indices_RS), len(remain_indices_RS))

  train_dataset = torch.utils.data.dataset.Subset(dataset, train_indices)
  remain_dataset = torch.utils.data.dataset.Subset(dataset, remain_indices)
  train_dataset_RS = torch.utils.data.dataset.Subset(dataset, train_indices_RS)
  remain_dataset_RS = torch.utils.data.dataset.Subset(dataset, remain_indices_RS)
  test_dataset = torch.utils.data.dataset.Subset(dataset, test_indices)

  # test MAE based on expanded training set
  def test_MAE(dataset, train_dataset, test_dataset):
    seed = 0
    batch_size = 100
    model = MEGNet(dataset).to('cuda')
    optimizer = getattr(torch.optim, "AdamW")(
        model.parameters(),
        lr=0.0005
    )
    scheduler = getattr(torch.optim.lr_scheduler, "ReduceLROnPlateau")(
        optimizer,**{"mode":"min", "factor":0.8, "patience":10, "min_lr":0.00001, "threshold":0.0002}
    )
    train_ratio = 0.95
    val_ratio = 0.05
    dataset_size = len(train_dataset)
    train_length = int(dataset_size * train_ratio)
    val_length = int(dataset_size * val_ratio)
    unused_length = dataset_size - train_length - val_length
    (
        train_dataset_for_testing,
        val_dataset,
        unused_dataset,
    ) = torch.utils.data.random_split(
        train_dataset,
        [train_length, val_length, unused_length],
        generator=torch.Generator().manual_seed(seed),
    )
    train_loader = DataLoader(
            train_dataset_for_testing,
            batch_size=batch_size)
    val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size)
    test_loader = DataLoader(
        test_dataset,
        batch_size = batch_size)
    model = trainer('cuda',1,model,optimizer,scheduler,'l1_loss', train_loader, val_loader, None, 250, 5,"my_model_temp.pth")
    test_error, test_out = evaluate(
                    test_loader, model, 'l1_loss', 'cuda', out=True
                )
    return test_error

  test_error_AL = test_MAE(dataset, train_dataset, test_dataset)
  print('round ' + str(AL_rounds) + ' AL test error: ', test_error_AL.cpu().numpy())
  test_error_RS = test_MAE(dataset, train_dataset_RS, test_dataset)
  print('round ' + str(AL_rounds) + ' RS test error: ', test_error_RS.cpu().numpy())
  all_AL_test_errors.append(float(test_error_AL.cpu().numpy()))
  all_RS_test_errors.append(float(test_error_RS.cpu().numpy()))

  # create cv datasets for AL uncertainty quantification
  num_folds = 5
  dataset_size = len(train_dataset)
  fold_length = int(dataset_size / num_folds)
  unused_length = dataset_size - fold_length * num_folds
  folds = [fold_length for i in range(num_folds)]
  folds.append(unused_length)
  cv_dataset = torch.utils.data.random_split(
      train_dataset, folds, generator=torch.Generator().manual_seed(seed)
  )
  cv_dataset = cv_dataset[0:num_folds]
  
  # predict uncertainty on remaining data and choose highest uncertainty materials to add to training set
  remain_loader = DataLoader(remain_dataset, batch_size=batch_size)
  remain_preds_df_all = None
  for index in range(0,(len(cv_dataset))):
      train_dataset = [x for i, x in enumerate(cv_dataset) if i != index]
      train_dataset = torch.utils.data.ConcatDataset(train_dataset)
      val_dataset = cv_dataset[index]
      print(len(train_dataset), len(test_dataset))
      train_loader = DataLoader( train_dataset, batch_size=batch_size)
      val_loader = DataLoader(val_dataset, batch_size=batch_size)
      model = MEGNet(dataset).to('cuda')
      optimizer = getattr(torch.optim, "AdamW")(
          model.parameters(),
          lr=0.0005
      )
      scheduler = getattr(torch.optim.lr_scheduler, "ReduceLROnPlateau")(
          optimizer,**{"mode":"min", "factor":0.8, "patience":10, "min_lr":0.00001, "threshold":0.0002}
      )
      model = trainer('cuda',1,model,optimizer,scheduler,'l1_loss', train_loader, val_loader, None, 50, 5,"my_model_temp.pth")
      remain_error, remain_out = evaluate(
                  remain_loader, model, 'l1_loss', 'cuda', out=True
              )
      remain_preds = [[x[0],float(x[2])] for x in remain_out]
      remain_preds_df = pd.DataFrame(remain_preds,columns=('filename','pred'+str(index)))
      if remain_preds_df_all is None:
        remain_preds_df_all = remain_preds_df
      else:
        remain_preds_df_all = pd.merge(remain_preds_df_all, remain_preds_df, on='filename', how='inner')

  remain_preds_df_all['unc'] = remain_preds_df_all.std(axis=1,numeric_only=True)
  remain_preds_df_all.sort_values('unc',ascending=False, inplace=True)
  add_AL_filenames = pd.DataFrame(remain_preds_df_all['filename'][:500])
  
  # randomly choose 500 materials to add to RS training set
  add_RS_filenames = pd.DataFrame(df_remain_RS[0].sample(500))

  # repeat!
  AL_rounds += 1


2576 4784 11189
2576 4784 11189


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.23527, Val Error: 1.21334, Time per epoch (s): 2.44364
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.91370, Val Error: 1.17519, Time per epoch (s): 2.11840
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.74425, Val Error: 1.01151, Time per epoch (s): 2.02979
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.67382, Val Error: 0.91712, Time per epoch (s): 2.05284
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.61410, Val Error: 0.98435, Time per epoch (s): 2.09936
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.53644, Val Error: 0.87964, Time per epoch (s): 2.29524
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.49545, Val Error: 0.88058, Time per epoch (s): 2.05278
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.47827, Val Error: 0.85941, Time per epoch (s): 2.09267
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.45285, Val Error: 0.88742, Time per epoch (s): 2.05900
E

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.29577, Val Error: 1.26055, Time per epoch (s): 2.25242
Epoch: 0010, Learning Rate: 0.000500, Training Error: 1.01529, Val Error: 1.17141, Time per epoch (s): 2.05879
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.83044, Val Error: 1.10235, Time per epoch (s): 1.94978
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.71434, Val Error: 1.04205, Time per epoch (s): 2.02204
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.62501, Val Error: 1.12728, Time per epoch (s): 2.15699
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.58856, Val Error: 1.00514, Time per epoch (s): 1.91379
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.51491, Val Error: 0.95105, Time per epoch (s): 1.95865
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.50909, Val Error: 0.97728, Time per epoch (s): 2.99437
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.46554, Val Error: 0.88924, Time per epoch (s): 2.16224
E

...

Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.24046, Val Error: 1.32856, Time per epoch (s): 3.98229
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.98427, Val Error: 1.12403, Time per epoch (s): 4.06748
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.85444, Val Error: 1.06343, Time per epoch (s): 4.19965
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.77918, Val Error: 1.02806, Time per epoch (s): 3.91525
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.71336, Val Error: 0.99293, Time per epoch (s): 4.37720
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.64827, Val Error: 1.14153, Time per epoch (s): 3.94697
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.61528, Val Error: 0.96393, Time per epoch (s): 4.11824
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.55787, Val Error: 0.98971, Time per epoch (s): 4.02783
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.56453, Val Error: 1.07598, Time per epoch (s): 4.04242
E

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.30659, Val Error: 1.47109, Time per epoch (s): 4.14265
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.99399, Val Error: 1.31077, Time per epoch (s): 4.24626
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.85033, Val Error: 1.06538, Time per epoch (s): 4.01693
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.75132, Val Error: 1.04092, Time per epoch (s): 4.13547
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.66876, Val Error: 1.13329, Time per epoch (s): 4.06360
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.63652, Val Error: 0.92869, Time per epoch (s): 4.02314
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.60093, Val Error: 0.93296, Time per epoch (s): 4.37916
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.57059, Val Error: 1.01505, Time per epoch (s): 3.94233
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.54961, Val Error: 0.99511, Time per epoch (s): 4.23268
E

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.11379, Val Error: 1.20674, Time per epoch (s): 4.45364
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.89158, Val Error: 1.18288, Time per epoch (s): 4.68019
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.81578, Val Error: 1.10720, Time per epoch (s): 5.47223
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.73153, Val Error: 1.06094, Time per epoch (s): 4.55587
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.68225, Val Error: 1.10907, Time per epoch (s): 4.34153
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.63279, Val Error: 1.37664, Time per epoch (s): 4.61389
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.59394, Val Error: 1.09683, Time per epoch (s): 4.35614
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.55976, Val Error: 0.93173, Time per epoch (s): 4.36548
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.51735, Val Error: 0.97936, Time per epoch (s): 4.52847
E

In [ ]:
print(all_AL_test_errors)
print(all_RS_test_errors)

In [ ]:
plt.figure(num=1,clear=True)
AL_rounds_x = np.arange(0,len(all_AL_test_errors))
plt.plot(AL_rounds_x, all_AL_test_errors, label='active learning')
plt.plot(AL_rounds_x, all_RS_test_errors, label='random sampling')
plt.xlabel('AL Round')
plt.ylabel('Fixed test set MAE')

In [ ]:
df_train.to_csv('/content/drive/MyDrive/Colab Notebooks/ML4ME/project/3.C51 Project/AL_training_final.csv')
df_train_RS.to_csv('/content/drive/MyDrive/Colab Notebooks/ML4ME/project/3.C51 Project/RS_training_final.csv')